In [1]:
import numpy as np
from tqdm import tqdm
from constants import MAX_SEQ_LEN

with open('../hw_train.csv', 'rb') as f:
    ls = f.readlines()[1:]
# count vocabulary and max sequence length
lyrics_map = {}
bos_token = 's'.decode('utf8')
eos_token = 'e'.decode('utf8')
none_lyrics_words = [u'編詞', u'作曲', u'作詞', u'編曲', u'監製']
vocab = [bos_token, eos_token]
lens = []
c = 0 
with tqdm(total=len(ls)) as pbar:
    for l in ls:
        l = l.decode('utf8')
        no, l = l.strip().split(',')
        no = int(no)
        if no not in lyrics_map:
            lyrics_map[no] = []
        lyrics_map[no].append(l)
        pbar.update(1)
# split into Q,K pair
with tqdm(total=len(lyrics_map)) as pbar:
    for k,v in lyrics_map.items():
        line_num = len(v)
        V = [bos_token,]
        Q = [bos_token,]
        for i,l in enumerate(v):
            exist_none_layrics_word = False
            for nw in none_lyrics_words:
                if nw in l:
                    exist_none_layrics_word = True
                    break
            if exist_none_layrics_word:
                continue
            for ws in l.strip().split(' '):
                for w in ws:
                    if i <= line_num//2:
                        V.append(w)
                    else:
                        Q.append(w)
                    vocab.append(w)
        Q.append(eos_token)
        V.append(eos_token)
        lens.append(len(Q))
        lens.append(len(V))
        lyrics_map[k] = (Q, V)
        pbar.update(1)
            
print 'vocabulary size : %d, max len : %.0f, mean : %.0f, std : %.0f' % (len(set(vocab)), np.max(lens), np.mean(lens), np.std(lens))
print 'lyrics number : %d' % len(lyrics_map)
# print np.max(lens), np.mean(lens), np.std(lens)



100%|██████████| 18338/18338 [00:02<00:00, 7287.33it/s]


vocabulary size : 6455, max len : 3575, mean : 161, std : 106
lyrics number : 18338


In [2]:
import numpy as np
import keras
from constants import MAX_SEQ_LEN, VOCAB_DIM
from keras.preprocessing.sequence import pad_sequences

texts = vocab

tokenizer = keras.preprocessing.text.Tokenizer(num_words=VOCAB_DIM, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=False, split=' ', char_level=True, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Using TensorFlow backend.


Found 6455 unique tokens.


In [3]:
from constants import MAX_SEQ_LEN
from keras.preprocessing.sequence import pad_sequences

idx = np.random.permutation(len(lyrics_map))
val_num = len(lyrics_map)//10
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[val_num:]}
val_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, lyrics_map, max_seq_len, tokenizer):
    G = boostrap_generator(lyrics_map, max_seq_len, tokenizer)
    while True:
        Q = []
        K = []
        K_len = []
        for i in range(batch_size):
            q, k, k_len = next(G)
            Q.append(q)
            K_len.append(k_len)
            K.append(k)
            
        yield np.vstack(Q), np.vstack(K), np.vstack(K_len)
def boostrap_generator(lyrics_map, max_seq_len, tokenizer):
    while True:
        keys = lyrics_map.keys()
        for idx in np.random.permutation(len(lyrics_map)):
            no = keys[idx]
            Q, K = lyrics_map[no]
            
            Q = tokenizer.texts_to_sequences([Q, ])
            Q = pad_sequences(Q, maxlen=max_seq_len, padding='post', truncating='post')
            K = tokenizer.texts_to_sequences([K, ])
            K_len = len(K)
            K = pad_sequences(K, maxlen=max_seq_len, padding='post', truncating='post')
            
            yield Q, K, K_len
    
bat = 4
G = batch_boostrap_generator(bat, train_u_map, MAX_SEQ_LEN, tokenizer)
val_G = batch_boostrap_generator(bat, val_u_map, MAX_SEQ_LEN, tokenizer)
q, k, k_len = next(G)
print q.shape, k.shape, k_len.shape


(4, 4500) (4, 4500) (4, 1)


### Attention

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_normal_
from constants import VOCAB_DIM, D_MODEL, MAX_SEQ_LEN
# construct neuron network
class Seq2seq(nn.Module):

    def __init__(self, dm, num_lay):
        super(Seq2seq, self).__init__()
#         for construct cache positional encoding matrix.
        self.emb = nn.Embedding(VOCAB_DIM+1, dm, padding_idx=0)
        self.encoder = nn.GRU(dm, dm, num_lay, batch_first=True)
        self.decoder = nn.GRU(dm, dm, num_lay, batch_first=True)
        self.linear = nn.Linear(dm, VOCAB_DIM+1)

    def forward(self, Q, K, K_len):
        K = self.emb(K)
        Q = self.emb(Q)
        en_out, _ = self.encoder(K) 
#         catch last valid hidden vector
        batch,_ = K_len.size()
        h_out = []
        for i in range(batch):
            k_len = K_len[i,0]
            h_out.append(en_out[i:i+1,k_len,:])
        h_out = torch.cat(h_out, dim=0).unsqueeze(0)
#         decoder
        Q, _ = self.decoder(Q, h_out)
    
        batch, q_len, _ = Q.size()
        att_in = torch.cat([en_out, Q], dim=1)
        Q_mask = self.Q_mask_matrix(batch, q_len)
        K_mask = self.K_mask_matrix(batch, q_len, K_len, K.size()[1])
        
        mask = torch.cat([K_mask, Q_mask], dim=-1)
        out = self.dot_attention(Q, att_in, att_in, mask=mask)
        
        
#         out,_  = self.decoder(Q, h_out)
        
        y = self.linear(out)
        return y
    def Q_mask_matrix(self, batch, Q_len):
#         ByteTensor
        mask = torch.zeros([1, Q_len, Q_len], dtype=torch.uint8, requires_grad=False)
        for i in range(Q_len):
            mask[0,i,i+1:] = 1
        return mask.repeat(batch,1, 1).cuda()
    def K_mask_matrix(self, batch, Q_len, K_len, K_max_len):
        assert len(K_len.size()) == 2
#         ByteTensor
        mask = torch.zeros([batch, Q_len, K_max_len], dtype=torch.uint8, requires_grad=False)
        for i in range(batch):
            k_len = K_len[i,0]
            mask[i, :, k_len+1:] = 1
        return mask.cuda()
    
    def dot_attention(self, Q, K, V, mask):
        assert Q.size()[-1] == K.size()[-1]
        assert len(Q.size()) == 3 and len(K.size()) == 3 and len(V.size()) == 3
        out = torch.matmul(Q,K.permute(0,2,1))
        if mask is not None:
            out.masked_fill_(mask, -float('inf'))
        return torch.matmul(F.softmax(out, dim=-1), V)


bat = 7
lay_num = 1
model = Seq2seq(D_MODEL, lay_num).cuda()
# print(o.size())
q, k, k_len = next(G)
q = torch.LongTensor(q).cuda()
k = torch.LongTensor(k).cuda()
k_len = torch.LongTensor(k_len).cuda()
# Q = torch.randint(VOCAB_DIM+1, [bat, MAX_SEQ_LEN]).cuda()
# K = torch.randint(VOCAB_DIM+1, [bat, MAX_SEQ_LEN]).cuda()

o = model(q[:,:200], k, k_len)
print(o.size())
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))



torch.Size([2, 200, 6456])
1857336


In [ ]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%s<split>%s<split>%s<split>%s\n' % (n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label)
    log_file_stream.write(log_text.encode('utf8'))
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
def normal_acc(pred, label):
    pred = torch.argmax(pred, dim=-1)
    mask = torch.ones_like(label, dtype=torch.uint8)
#     
    acc = pred == label
    acc = torch.sum(acc).item() / float(torch.sum(mask).item())
    
    return acc
def seq2text(output, index_word):
    assert len(output.size()) == 1
    seq = output
    s = ''
    for i in seq:
        i = int(i.item())
        if i == 0:
            continue
        w = index_word[i]
        s += w
    return s
    
def rev_mask(m):
    out = torch.ones_like(m, dtype=torch.uint8, requires_grad=False)
    out.masked_fill_(m, 0)
    return out
        
acc_q = deque(maxlen=100)
loss_q = deque(maxlen=10)

val_acc_q = deque(maxlen=100)
val_loss_q = deque(maxlen=10)

t = time.time()
best_loss = float('inf')

epochs = 100
batch_size = 2
G = batch_boostrap_generator(batch_size, train_u_map, MAX_SEQ_LEN, tokenizer)
val_G = batch_boostrap_generator(batch_size, val_u_map, MAX_SEQ_LEN, tokenizer)
# criterion = nn.CrossEntropyLoss(reduction='none')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
print 'start training.'
with open('log-att.txt', 'w') as f:
    with open('best-att.txt', 'w') as best_log:
        iters = 100000000
        with tqdm(total=iters) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                q, k, k_len = next(G)
                q = torch.LongTensor(q).cuda()
                k = torch.LongTensor(k).cuda()
                k_len = torch.LongTensor(k_len).cuda()

                q.requires_grad_(False)
                k.requires_grad_(False)
                k_len.requires_grad_(False)


                output = model(q, k, k_len)
                y = k[:,1:]
#                 loss = torch.sum(criterion(output, y) * pad_mask[:,1:, :]) 
                loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                label = y

                pred = output[:,:-1,:,]
                acc = normal_acc(pred, label)
                acc_q.append(acc)
                train_pred = seq2text(torch.argmax(output[0,:,:], dim=-1), tokenizer.index_word)
                train_label= seq2text(k[0,:], tokenizer.index_word)
                loss.backward()
                loss_q.append(loss.item())
                
                optimizer.step()
                with torch.no_grad():
                    model.eval()
                    q, k, k_len = next(val_G)
                    q = torch.LongTensor(q).cuda()
                    k = torch.LongTensor(k).cuda()
                    k_len = torch.LongTensor(k_len).cuda()
                    
                    q.requires_grad_(False)
                    k.requires_grad_(False)
                    k_len.requires_grad_(False)
                    
                    output = model(q, k, k_len)
                    y = k[:,1:]
                    val_loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                    label = y

                    pred = output[:,:-1,:,]
                    val_acc = normal_acc(pred, label)
                    val_acc_q.append(val_acc)
                    val_pred = seq2text(torch.argmax(output[0,:,:], dim=-1), tokenizer.index_word)
                    val_label= seq2text(k[0,:], tokenizer.index_word)
                    val_loss_q.append(val_loss.item())


                
                acc = np.mean(acc_q)
                val_acc = np.mean(val_acc_q)
                loss = np.mean(acc_q)
                val_loss = np.mean(val_acc_q)
                
#                     pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f \t %.3f, %.3f, %.3f, %.3f' % (acc, val_acc, loss.item(), val_loss.item(), a,b,c,d), refresh=False)
                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f' % (acc, val_acc, loss.item(), val_loss.item()), refresh=False)
                pbar.update(batch_size)
                dump_log(model, (it+1)*batch_size, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, f,'./tmp-att.pt')
                if val_loss.item() < best_loss and it > 100:
                    torch.save(model, './best-att.pt')
                    best_loss = val_loss
                    best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_loss))
                    best_log.flush()
                if it % 100 == 0 and it >= 100: 
                    print 'train pred : %s\ntrain label : %s' % (train_pred, train_label)
                    print 'validation pred : %s\nvalidation label : %s' % (val_pred, val_label)

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/100000000 [00:00<?, ?it/s]

start training.


  0%|          | 202/100000000 [01:51<14819:33:36,  1.87it/s, acc : 0.936, val_acc : 0.946, loss : 0.936, val_loss : 0.946]

train pred : 
train label : s编曲金若晨韦国赟吉他金东昱和声编写韦国赟和声陈敏韦国赟童声配唱录音师郝宇配唱录音室喷气机录音棚混音师周天澈母带工程师周天澈愿你和我不再孤独和迷惘远离伤痛在蓝天下成长愿这大地处处有爱的阳光点亮我们的希望这是我的愿望也是你的梦想我们一路把歌唱这是我的梦想守护在你身旁有爱就会有希望愿每个人收获快乐和健康每张面孔都有笑容绽放愿这份爱在心底蔓延生长e
validation pred : 
validation label : s已经分手很久留下我们一起养的狗抱着它不寂寞彷佛你还宠着我真希望你懂我就像狗狗对我的温柔讨好不用太多贴心就够把我的手让它当作最温暖的小窝心里的话只对它说星空下勾小指头沿着幸福的路走喜欢和你逛宠物店买狗罐头狗狗日记的角落偷偷画着你和我三个影子拉长到永久懒洋洋的午后吐着帮我洗脸的舌头可爱的小毛球偶尔调皮爱闯祸日记里有太多和它一起生活的感受回家有它等我贴心守候现在的我和狗狗一起看夏日烟火e


  0%|          | 402/100000000 [03:38<14915:28:26,  1.86it/s, acc : 0.959, val_acc : 0.963, loss : 0.959, val_loss : 0.963]

...

  0%|          | 9202/100000000 [1:22:44<15239:07:05,  1.82it/s, acc : 0.965, val_acc : 0.963, loss : 0.965, val_loss : 0.963]

train pred : 的的的的我我我我我我的的的的的的的的的的的的的的的的的的的不不不不不不不不不不不不不不不我我我我我我我我我我我我我我你我你我我我我你你你我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我
train label : s又见雪飘过飘於伤心记忆中让我再想你却掀起我心痛早经分了手为何热爱尚情重独过追忆岁月或许此生不会懂又再想起你抱拥飘飘白雪中让你心中暖去驱走我冰冻冷风催我醒原来共你是场梦像那飘飘雪泪下弄湿冷清的晚空原来是那麽深爱你此际伴着我追忆的心痛又见雪飘过飘於伤心记忆中让我再想你e
validation pred : 我我我的的的的的的的我我我我我我我我我我我我的我我我我的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的一一一的的的的的
validation label : s灯已暗了旧州歌就要唱完了这是最后的音乐没有人需要鼓掌只剩下你来自异乡的姑娘坐在阁楼上想起你的名字我又醉了明天不会是末日没有人需要疯狂夜阑人静酒已尽只有我在这里唱没有公共汽车了通向你郊区的住房找个大排档一杯一杯到天亮没有公共汽车了通向你心中的梦想找个大排档一杯一杯到天亮没有公共汽车了通向你心中的爱情找个大排档一杯一杯到天亮没有公共汽车了通向你郊区的住房找个大排档一杯一杯到天亮明天不会是末日没有人需要疯狂e


  0%|          | 9402/100000000 [1:24:33<15210:58:28,  1.83it/s, acc : 0.964, val_acc : 0.959, loss : 0.964, val_loss : 0.959]

train pred : 的的的的你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你
train label : s静静伴你定定望你不禁神儿荡静静让你望住我将眼神释放春天望你夏夜望你都叫人神荡望罢让你望实我仔细地看伴白云白雪月或圆月缺只有你好看沙中望你风中望我一看再看静静伴你定定望你不禁神儿荡静静让你望住我将眼神释放e
validation pred : 我我的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的爱爱爱的的的的的的的的的的的的的的的的的的的的的的的的爱爱不爱爱爱爱爱爱爱爱爱爱爱的爱爱爱爱爱爱的的的的的的的爱的的的的的的的的的的的的的的的的的的的的的
validation label : s挥别过去所有阴霾冷风中那首歌就让桑田变成沧海一旦踏上这个舞台就算撑到最后一秒坚持不愿倒下来是潇洒是疯狂都是热血澎湃让怒吼让等待都变成喝彩谁胜出谁失败今朝拭目以待英雄用他的一生换最后的一声痛快血汗泪水苦尽甘来转身狂笑无尽风采千年之后若化作尘埃是英雄名不换姓不改挥别过去所有阴霾冷风中那首歌就让桑田变成沧海一旦踏上这个舞台就算撑到最后一秒坚持不愿倒下来是潇洒是疯狂都是热血澎湃e


  0%|          | 9414/100000000 [1:24:39<14985:14:42,  1.85it/s, acc : 0.965, val_acc : 0.959, loss : 0.965, val_loss : 0.959]